In [1]:
import json
import gzip
import numpy as np
from copy import deepcopy

In [2]:
def get_json_list_from_gz_file(file_name):
    json_list = []
    
    with gzip.open(file_name,'r') as fin:
        for line in fin:        
            data_json = json.loads(line.decode('utf-8'))
            json_list.append(data_json)
    
    return json_list

In [3]:
class CrawledData:
    
    def __init__(self, category, crawl_date, subcategory, title, mrp, urlh, http_status, pack_size, available_price):
        self.category = category
        self.crawl_date = crawl_date
        self.subcategory = subcategory
        self.title = title
        self.mrp = mrp
        self.urlh = urlh
        self.http_status = http_status
        self.pack_size = pack_size
        self.available_price = available_price
        
    @staticmethod
    def build_object(json_item):
        crawled_data = CrawledData(
            category=json_item['category'],
            crawl_date = json_item['crawl_date'],
            subcategory = json_item['subcategory'],
            title = json_item['title'],
            mrp = float(json_item['mrp'] or 0),
            urlh = json_item['urlh'],
            http_status = json_item['http_status'],
            pack_size = json_item['pack_size'],
            available_price = float(json_item['available_price'] or 0),
        )
        return crawled_data

class DataBuilder:
    
    @staticmethod
    def build_id_obj_dict(json_list):
        data_dict = {}
        
        for json_item in today_json_list:
            crawled_data = CrawledData.build_object(json_item)
            urlh = crawled_data.urlh

            if urlh in data_dict:
                if data_dict.get(urlh).http_status != '200' and crawled_data.http_status == '200':
                    data_dict[crawled_data.urlh] = crawled_data

            else:
                data_dict[crawled_data.urlh] = crawled_data
        
        return data_dict
    
    @staticmethod
    def build_cat_subcat_dict(id_obj_dict):
        cat_subcat_dict = {}
        for id, obj in id_obj_dict.items():
            if cat_subcat_dict.get(obj.category) is None:
                cat_subcat_dict[obj.category] = {}
                
            category_dict = cat_subcat_dict[obj.category]
            
            if category_dict.get(obj.subcategory) is None:
                category_dict[obj.subcategory] = list()
            
            subcat_list = category_dict.get(obj.subcategory)
            subcat_list.append(obj)
            
        return cat_subcat_dict


In [4]:
today_json_list = get_json_list_from_gz_file('t.json.gz')
yesterday_json_list = get_json_list_from_gz_file('y.json.gz')


In [5]:
todays_crawled_data_dict = DataBuilder.build_id_obj_dict(today_json_list)
yesterdays_crawled_data_dict = DataBuilder.build_id_obj_dict(yesterday_json_list)

In [6]:
# Q1. No. of overlapping URLHs.
''' Overlapping URLHs, occurring in both today and yesterday's catalog is found using set intersection.
    Set 1: URLHs of today's items.
    Set 2: URLHs of yesterday's items.
    The intersection yields the urlh ids of items common in both.
    
'''
overlapping_urlhs = set(todays_crawled_data_dict.keys()).intersection(set(yesterdays_crawled_data_dict.keys()))
print("Number of overlapping items is ", len(overlapping_urlhs))

Number of overlapping items is  131489


In [7]:
# Q2. Price difference between overlapping URLHS.
'''Using the list of overlapping urlhs from previous calculation, difference in price can be calculated.
'''
price_diff_dict = {}
for urlh in overlapping_urlhs:
    price_diff = todays_crawled_data_dict.get(urlh).available_price - yesterdays_crawled_data_dict.get(urlh).available_price
    if price_diff != 0:
        price_diff_dict[urlh] = price_diff

print("Number of items with difference in price: ", len(price_diff_dict))
price_diff_dict

Number of items with difference in price:  0


{}

In [8]:
todays_cat_subcat_dict = DataBuilder.build_cat_subcat_dict(todays_crawled_data_dict)
yesterdays_cat_subcat_dict = DataBuilder.build_cat_subcat_dict(yesterdays_crawled_data_dict)

In [9]:
# Q3 Number of unique categories in both files
'''Since, the keys of dictionaries are unique, the number of keys in the category-subcategory dict gives the number of unique categories.
'''
no_of_uniques_categories_in_todays_dict = len(todays_cat_subcat_dict.keys())
no_of_uniques_categories_in_yesterdays_dict = len(yesterdays_cat_subcat_dict.keys())
print("Unique categories in today's file: ", no_of_uniques_categories_in_todays_dict)
print("Unique categories in yesterdays's file: ", no_of_uniques_categories_in_yesterdays_dict)


Unique categories in today's file:  50
Unique categories in yesterdays's file:  50


In [10]:
# Q4 Set of categories which don't have any overlapping items.
'''Compute the global categories set and remove the categories of overlapping items from it.
'''
categories = set(todays_cat_subcat_dict.keys()).union(set(yesterdays_cat_subcat_dict.keys()))
categories_of_overlapping_items = {todays_crawled_data_dict[urlh].category for urlh in overlapping_urlhs}

non_overlapping_categories =  categories - categories_of_overlapping_items
print("Non overlapping categories: ", non_overlapping_categories)

Non overlapping categories:  set()


In [11]:
# Q5 Taxonomy generation
'''Taxonomy stats can be generated iterating through the category-subcategory dict.
'''
def generate_taxonomy_details(cat_subcat_dict, query_func):
    '''A function which generates taxonomy stats as required by the query func.
    Args: 
        cat_subcat_dict: The category-subcategory dict.
        query_func: The function which represents the query. In this example, len is passed to get the number of items in the list
        corresponding to that subcategory. Other functions corresponding to average, median, etc of any other metric of the list
        can also be passed.
    
    Returns:
         taxonomy_stats: Dict representing the queried stats.
    '''
    taxonomy_stats = {}
    for category in cat_subcat_dict:
        subcat_dict = cat_subcat_dict[category]
        for subcat in subcat_dict:
            item_list = subcat_dict[subcat]
            print(category, " --> ", subcat, " --> ", query_func(item_list))
            taxonomy_stats[(category, subcat)] = query_func(item_list)
    
    return taxonomy_stats

todays_taxonomy_stats = generate_taxonomy_details(todays_cat_subcat_dict, len)
yesterdays_taxonomy_stats = generate_taxonomy_details(yesterdays_cat_subcat_dict, len)

Household  -->  Trash Bags & Liners  -->  175
Household  -->  Paper Goods  -->  584
Household  -->  Laundry  -->  1051
Household  -->  Kitchen Supplies  -->  1155
Household  -->  Plates, Bowls, Cups & Flatware  -->  486
Household  -->  Food Storage  -->  615
Household  -->  Dish Detergents  -->  388
Household  -->  Cleaning Products  -->  1505
Household  -->  Air Fresheners & Candles  -->  807
Household  -->  More Household  -->  3900
Household  -->  Garden  -->  56
Household  -->  Books & Magazines  -->  1
Pets  -->  Dog Food & Care  -->  1493
Pets  -->  Cat Food & Care  -->  1233
Pets  -->  Small Animal Care  -->  153
Babies  -->  Baby Accessories  -->  236
Babies  -->  Baby First Aid & Vitamins  -->  197
Babies  -->  Diapers & Wipes  -->  542
Babies  -->  Baby Food & Formula  -->  1259
Babies  -->  Baby Bath & Body Care  -->  264
Personal Care  -->  Adult Care  -->  66
Personal Care  -->  Foot Care  -->  138
Personal Care  -->  Family Planning  -->  241
Personal Care  -->  Aromather

In [24]:
!pip install plotly
import plotly.plotly as py
import plotly.graph_objs as go

py.sign_in('PythonAPI', 'ubpiol2cve')

You are using pip version 18.0, however version 19.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [25]:
'''Plots the taxonomy stats in plotly for a given category'''
import plotly.plotly as py
import plotly.graph_objs as go

category_to_plot = 'Household'

today = go.Bar(
    x=[cat_subcat[1] for cat_subcat in todays_taxonomy_stats if cat_subcat[0] == category_to_plot],
    y= [value for key, value in todays_taxonomy_stats.items() if key[0] == category_to_plot],
    name='''Today's taxonomy stats''',
    marker=dict(
        color='rgb(55, 83, 109)'
    )
)
yesterday = go.Bar(
    x=[cat_subcat[1] for cat_subcat in todays_taxonomy_stats if cat_subcat[0] == category_to_plot],
    y= [value for key, value in todays_taxonomy_stats.items() if key[0] == category_to_plot],
    name='''Yesterdays's taxonomy stats''',
    marker=dict(
        color='rgb(26, 118, 255)'
    )
)
data = [today, yesterday]
layout = go.Layout(
    title= category_to_plot + ' ' 'taxonomy stats',
    xaxis=dict(
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis=dict(
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='style-bar')

In [26]:
# Q6 Normalized MRP
'''MRP can be normalized such that the mean in 0 and std dev is 1.
'''

mrp = np.array([obj.mrp for obj in todays_crawled_data_dict.values() if obj.mrp != 0])
mrp_mean = np.mean(mrp)
mrp_std = np.std(mrp)
mrp_normalized_item_list = []

for item in todays_crawled_data_dict.values():
    item_copy = deepcopy(item)
    
    if item_copy.mrp == 0:
        item_copy.mrp = 'NA'
    
    else:
        item_copy.mrp = (item_copy.mrp - mrp_mean) / mrp_std
    
    mrp_normalized_item_list.append(item_copy.__dict__)
    
json_str = json.dumps(mrp_normalized_item_list) + "\n"               
json_bytes = json_str.encode('utf-8')            

with gzip.GzipFile('t_mrp_normalized.json.gz', 'w') as fout: # Write to a output file
    fout.write(json_bytes)             